In [15]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_4.csv",r"B:\project\ai\ai_chunk_4.csv","B:\project\human\human_chunk_5.csv",r"B:\project\ai\ai_chunk_5.csv","B:\project\human\human_chunk_6.csv",r"B:\project\ai\ai_chunk_6.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(91740, 2)


In [17]:
from transformers import LongformerTokenizer
data=df['text'].tolist()
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokens=tokenizer(data,return_tensors='pt',padding=True,truncation=True)

In [5]:
tokens

tensor([[0, 3, 3,  ..., 3, 3, 2]])

In [21]:
from sklearn.model_selection import train_test_split
x=tokens['input_ids']
y=df['generated']
xtrain,xtemp,ytrain,ytemp=train_test_split(x,y,test_size=0.3,random_state=42)
xtest,xval,ytest,yval=train_test_split(xtemp,ytemp,test_size=0.3,random_state=42)
xtest,xtrain,xval=xtest.numpy(),xtrain.numpy(),xval.numpy()

In [22]:
from keras.models import load_model
model=load_model('finetuned1.keras')
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=5,batch_size=256,validation_data=(xval,yval))

Epoch 1/5
251/251 [==============================] - 136s 538ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.0040 - val_accuracy: 0.9993
Epoch 2/5
251/251 [==============================] - 149s 592ms/step - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.0044 - val_accuracy: 0.9990
Epoch 3/5
251/251 [==============================] - 146s 582ms/step - loss: 6.5534e-04 - accuracy: 0.9999 - val_loss: 0.0034 - val_accuracy: 0.9995
Epoch 4/5
251/251 [==============================] - 145s 579ms/step - loss: 5.4772e-04 - accuracy: 1.0000 - val_loss: 0.0048 - val_accuracy: 0.9990
Epoch 5/5
251/251 [==============================] - 146s 581ms/step - loss: 1.5332e-04 - accuracy: 1.0000 - val_loss: 0.0041 - val_accuracy: 0.9994


In [23]:
model.evaluate(xtest,ytest)

603/603 [==============================] - 9s 15ms/step - loss: 0.0030 - accuracy: 0.9991


[0.0030358871445059776, 0.9991175532341003]

In [25]:
model.save('finetuned2.keras')

In [26]:
model1=load_model('finetuned2.keras')

In [36]:
# x='My argument for the development of the driverless cars is that we should have them These cars should be allowed to be made they can become really helpful to people They can also provide really good safety features The diverless car will help the environment it will spend less of our natrual resources that we use These cars will change the way our future will look like First of all the diverless cars will help tons of people around the world Think of all the handicap people on this earth who can t drive These cars will change their lives They can travel by their selfs instead of people picking them up to go places There might be people who become drunk or injured and they need to go to their house or hopistal With these cars they can get there Second the driverless car can provide really good safety features The cars will have a lot of sensors When Google modifed their Toyota Prius they use position estimating senosors rotating sensors video cameras four automotive radar sensors etc They put all theses thing on a car to make it safe and to drive on its own The driverless car can have all these features to be a very safe car to drive Last the driverless cars will help the environment and will spend less of our natrual resources A Google cofounder Sergey Brin says that the cars he foresees would use half the fuel of taday s taxis and offer far more flexiblity than a bus These driverless cars provide less car accidents With less accidents people won t get killed or injured The driverless car will aslo provide less money of fuel In conclusion the driverless cars will change our future It will help people get around easily The cars will provide greay safety features Its will also help the environment I hope i would be able to see these driverless car around someday'
x='Thomas Jefferson was a prominent American statesman and Founding Father who is most famously remembered for writing the Declaration of Independence He was a passionate believer in hard work and the value of individuals coming together to serve each other and their country According to Jefferson respect and collaboration are integral parts of any successful society These beliefs of Jefferson s still resonate today especially in the modern workplace As technology continues to advance and more people are relying on collaborative talents it is essential to remember the lessons that Jefferson taught us about the need to work hard and treat others with respect People of all backgrounds need to come together to ensure that we are able to effectively and efficiently solve any issues that arise Of course mutual respect is key to this endeavor The sentiments that Jefferson promoted are as relevant now as they ve ever been We must never forget the importance of values such as hard work and respect for each other His unwavering commitment to ensuring the longevity of these principles can continue to serve us and help us move towards a brighter and more productive future'
tokens=tokenizer(x,return_tensors='pt',padding=True,truncation=True)
input=tokens['input_ids'].numpy()
pred=model1.predict(input)
threshold = 0.5
prediction_value = pred[0, 0]

# Check the prediction and print the result
if prediction_value >= threshold:
    print("1: It is an AI-generated text")
else:
    print("0: It is Human-generated text")

1/1 [==============================] - 0s 18ms/step
1: It is an AI-generated text
